# Basic Blockchain Architecture

#### Implementação da arquiteura básica de uma Blockchain, passando por conceitos envolvendo transações, blocos, validação e atualização do estado do sistema. Na maior parte da implementação são usadas contas de duas pessoas (Elcius e Johannes) como exemplificação do funcionamento básico. Para uma introdução teórica ao assunto recomendo [este artigo do site Bits On Blocks](https://bitsonblocks.net/2015/09/09/a-gentle-introduction-to-blockchain-technology/).

#### De forma simplificada, uma Blockchain é uma base de dados distribuída com um conjunto de regras para verificação da validade de novos elementos. Inicialmente, iremos acompanhar os saldos das contas de duas pessoas conforme simulamos a transferência de dinheiro entre elas. Um local para armazenamento de transações será criado para que elas sejam acessadas, validadas, inseridas em um bloco e para que esse bloco seja inserido ao final da Blockchain. 

#### Durante esse processo, várias verificações são feitas a fim de validar se as informações são honestas. Antes de se implementar essas verificações, é necessário trabalhar com funções que serão de alta importância, são elas as funções de hash.

### SHA256

#### Função auxiliar que utiliza o algoritmo SHA256 como referência para a criação do hash.

In [295]:
import hashlib, json, sys

''' Receives the message itself and returns the result of hash algoritm '''
def hashing(msg=""):
    if type(msg)!=str:
        msg = json.dumps(msg, sort_keys=True)   #loading json file to dictionary
    
    result = hashlib.sha256(str(msg).encode('utf-8')).hexdigest()
    
    return result

## Transações

### Gerando transações

#### Nas transações, o envio de dinheiro será considerado como um número negativo e o recebimento como positivo. As transações são criadas como sendo entre dois usuários (Elcius e Johannes) do sistema sem que haja criação ou destruição de dinheiro.

In [296]:
import random
random.seed(0)

''' Creates valid transactions in the range of 1 to max_value and returns them '''
def transaction_generator(max_value=3):
    sign = int(random.getrandbits(1)) * 2 - 1   # results in -1 or 1 randomly
    amount = random.randint(1, max_value)   # randomly choose the amount from 1 to max_value
    
    elcius_value = sign * amount
    johannes_value = - sign * amount
    
    summary = {'Elcius':elcius_value, 'Johannes':johannes_value}
    
    return summary

### Criando um conjunto de transações

In [297]:
trns_buffer = [transaction_generator() for i in range(15)]
trns_buffer

[{'Elcius': 2, 'Johannes': -2},
 {'Elcius': 2, 'Johannes': -2},
 {'Elcius': -2, 'Johannes': 2},
 {'Elcius': 3, 'Johannes': -3},
 {'Elcius': -2, 'Johannes': 2},
 {'Elcius': 2, 'Johannes': -2},
 {'Elcius': 2, 'Johannes': -2},
 {'Elcius': -3, 'Johannes': 3},
 {'Elcius': 1, 'Johannes': -1},
 {'Elcius': 1, 'Johannes': -1},
 {'Elcius': -1, 'Johannes': 1},
 {'Elcius': 1, 'Johannes': -1},
 {'Elcius': 2, 'Johannes': -2},
 {'Elcius': 3, 'Johannes': -3},
 {'Elcius': 3, 'Johannes': -3}]

#### Portanto, se Elcius está com valor -2 e Johannes com valor 2 na transação, significa que Elcius transferiu duas unidades de dinheiro ao usuário de Johannes.

### Validação de transações

#### Para realizar a validação, estabelecemos duas regras que se violadas rejeitarão a transação:

* A soma dos envios e dos recebimentos deve ser igual a zero
* Um usuário deve ter saldo suficiente para realizar envios de dinheiro

In [298]:
''' Receives a transaction and if it is valid returns True else returns False '''
def transaction_validation(trn, state):
# tr: dictionary with transaction, all keys are account names
# state: current balances of each account
    if sum(trn.values()) != 0:   # check if no money was created or destroyed
        return False

    for key in trn.keys():
        if key in state.keys():   # check if user exists at state dictionary
            account_balance = state[key]   # access current balance
        else: 
            account_balance = 0

        if (account_balance + trn[key]) < 0:   # check if user has money enough to send
            return False
    
    return True

#### Testando a validação, se resultar em 'True' a transação é válida, se resultar em 'False' é inválida.

In [299]:
state = {'Elcius':5, 'Johannes':5}   # Set account balances

for tr in trns_buffer:
    print(transaction_validation(tr, state), tr)

True {'Elcius': 2, 'Johannes': -2}
True {'Elcius': 2, 'Johannes': -2}
True {'Elcius': -2, 'Johannes': 2}
True {'Elcius': 3, 'Johannes': -3}
True {'Elcius': -2, 'Johannes': 2}
True {'Elcius': 2, 'Johannes': -2}
True {'Elcius': 2, 'Johannes': -2}
True {'Elcius': -3, 'Johannes': 3}
True {'Elcius': 1, 'Johannes': -1}
True {'Elcius': 1, 'Johannes': -1}
True {'Elcius': -1, 'Johannes': 1}
True {'Elcius': 1, 'Johannes': -1}
True {'Elcius': 2, 'Johannes': -2}
True {'Elcius': 3, 'Johannes': -3}
True {'Elcius': 3, 'Johannes': -3}


#### Outros casos de teste:

In [300]:
trn = {'Elcius': -4, 'Johannes': 3}
print('1st case:', transaction_validation(trn,state), trn)
trn = {'Elcius': -4, 'Johannes': 5}
print('2nd case:', transaction_validation(trn,state), trn)
trn = {'Elcius': -6, 'Johannes': 6}
print('3rd case:', transaction_validation(trn,state), trn)
trn = {'Elcius': -4, 'Johannes': 6,'Samuel': -2}
print('4th case:', transaction_validation(trn,state), trn)
trn = {'Elcius': -4, 'Johannes': 2,'Samuel':2}
print('5th case:', transaction_validation(trn,state), trn)
trn = {'Elcius': -4, 'Johannes': 3,'Samuel':2}
print('6th case:', transaction_validation(trn,state), trn)

1st case: False {'Elcius': -4, 'Johannes': 3}
2nd case: False {'Elcius': -4, 'Johannes': 5}
3rd case: False {'Elcius': -6, 'Johannes': 6}
4th case: False {'Elcius': -4, 'Johannes': 6, 'Samuel': -2}
5th case: True {'Elcius': -4, 'Johannes': 2, 'Samuel': 2}
6th case: False {'Elcius': -4, 'Johannes': 3, 'Samuel': 2}


* No primeiro caso o resultado foi 'False' e a transação foi invalidada pois, durante a mesma, 1 unidade de dinheiro foi perdida, o que não pode acontecer.
* O segundo caso resultou em 'False' e na invalidação pois, durante a transação, 1 unidade de dinheiro foi criada.
* No terceiro, o resultado foi 'False' e foi invalidada pois Elcius não possuía 6 unidades de dinheiro como saldo na conta.
* O segundo caso resultou em 'False' pois foi inserida uma pessoa na transação (Samuel) que não apresenta saldo em conta no dicionário 'state'.
* No quinto caso, o resultado foi 'True' e a transação foi validada pois não há problemas em inserir uma pessoa a mais, na transação, que esteja recebendo dinheiro, mesmo ela não estando presente inicialmente no dicionário _state_ junto das informações das outras contas.
* No sexto e último caso, a transação foi invalidada pois houve novamente a criação de 1 unidade de dinheiro ao se considerar os dois recebedores.

### Atualizando os saldos

#### Para cada nova transação o estado dos saldos deve ser atualizado.

In [301]:
''' Updates state dictionary and add users to it if necessary '''
def update_state(trn, state):
# trn: dictionary with transaction, all keys are account names
# state: current balances of each account
    state = state.copy()
    for key in trn:
        if key in state.keys():
            state[key] += trn[key]
        else:   # add new account
            state[key] = trn[key]
            
    return state

In [302]:
state = {'Elcius':5, 'Johannes':5}   # Set initial account balances

for trn in trns_buffer:
    if transaction_validation(trn,state):
        state = update_state(trn, state)
        print('Transaction: %s is VALID. Current State: %s' %(trn, state))
    else:
        print('Transaction: %s is INVALID. Current State: %s' %(trn, state))

Transaction: {'Elcius': 2, 'Johannes': -2} is VALID. Current State: {'Elcius': 7, 'Johannes': 3}
Transaction: {'Elcius': 2, 'Johannes': -2} is VALID. Current State: {'Elcius': 9, 'Johannes': 1}
Transaction: {'Elcius': -2, 'Johannes': 2} is VALID. Current State: {'Elcius': 7, 'Johannes': 3}
Transaction: {'Elcius': 3, 'Johannes': -3} is VALID. Current State: {'Elcius': 10, 'Johannes': 0}
Transaction: {'Elcius': -2, 'Johannes': 2} is VALID. Current State: {'Elcius': 8, 'Johannes': 2}
Transaction: {'Elcius': 2, 'Johannes': -2} is VALID. Current State: {'Elcius': 10, 'Johannes': 0}
Transaction: {'Elcius': 2, 'Johannes': -2} is INVALID. Current State: {'Elcius': 10, 'Johannes': 0}
Transaction: {'Elcius': -3, 'Johannes': 3} is VALID. Current State: {'Elcius': 7, 'Johannes': 3}
Transaction: {'Elcius': 1, 'Johannes': -1} is VALID. Current State: {'Elcius': 8, 'Johannes': 2}
Transaction: {'Elcius': 1, 'Johannes': -1} is VALID. Current State: {'Elcius': 9, 'Johannes': 1}
Transaction: {'Elcius': -

## Cadeia e Blocos

#### Para iniciar o trabalho com os blocos da Blockchain primeiro devemos criar o bloco inicial da cadeia, também conhecido como _genesis block_. Sejam os seguintes saldos iniciais das contas:

In [303]:
state = {'Elcius': 50, 'Johannes': 50}

### Genesis Block

#### Os saldos acima constituem o conteúdo que ficará armazenado como transações no bloco inicial. Por ele ser o primeiro, será tratado um pouco diferente em relação a um bloco normal.

In [304]:
genesis_block_trns = [state]

genesis_block_content = {
    'block_number': 0,
    'parent_hash': None,
    'trns_quantity': 1,
    'transactions': genesis_block_trns
}

genesis_hash = hashing(genesis_block_content)

genesis_block = {
    'hash': genesis_hash,
    'content': genesis_block_content
}

#### Agora podemos adicionar esse primeiro bloco à cadeia:

In [305]:
chain = [genesis_block] # chain represented by a list of blocks
print(json.dumps(chain, indent=3))

[
   {
      "hash": "b1f3f3cf7fb886cffc1e4b61cf65a74d18d2ee8b85ee80db26a8b0e6e2795ef5",
      "content": {
         "block_number": 0,
         "parent_hash": null,
         "trns_quantity": 1,
         "transactions": [
            {
               "Elcius": 50,
               "Johannes": 50
            }
         ]
      }
   }
]


### Criando novos blocos

#### Em relação à criação dos blocos, temos que para cada novo bloco um número vai ser atribuído, o hash do bloco anterior vai ser obtido e um conjunto de transações vai ser coletado juntamente com a quantidade das mesmas. O Hash do bloco será criado levando em consideração todas essas informações.

In [306]:
''' Create a new block and return it '''
def make_block(trns, chain):
# trns: set of transactions
# chain: the chain list
    parent_block = chain[-1]
    parent_hash = parent_block['hash']   
    block_number = parent_block['content']['block_number'] + 1   
    trns_quantity = len(trns)
    
    block_content = {
        'block_number': block_number,
        'parent_hash': parent_hash,
        'trns_quantity': trns_quantity,
        'transactions': trns
    }
    
    block_hash = hashing(block_content)
    
    block = {
        'hash': block_hash,
        'content': block_content
    }
    
    return block

#### O buffer de transações será processado por inteiro e novos blocos serão feitos contendo no máximo 5 transações.

In [307]:
block_size_limit = 5   # limit quantity of transactions per block

while len(trns_buffer) > 0:   # process entire buffer
    trns_list = []   # store 5 transactions at most
    
    while (len(trns_buffer) > 0) and (len(trns_list) < block_size_limit):
        new_trn = trns_buffer.pop()   # remove and get a transaction from buffer
        valid = transaction_validation(new_trn, state)   # validate
        
        if valid:
            trns_list.append(new_trn)
            state = update_state(new_trn, state)            
            print('Valid transaction: %s,  State: %s' %(new_trn, state))
        else:
            print('Invalid transaction: %s,  State: %s' %(new_trn, state))
        
    new_block = make_block(trns_list, chain)
    chain.append(new_block)

Valid transaction: {'Elcius': 3, 'Johannes': -3},  State: {'Elcius': 53, 'Johannes': 47}
Valid transaction: {'Elcius': 3, 'Johannes': -3},  State: {'Elcius': 56, 'Johannes': 44}
Valid transaction: {'Elcius': 2, 'Johannes': -2},  State: {'Elcius': 58, 'Johannes': 42}
Valid transaction: {'Elcius': 1, 'Johannes': -1},  State: {'Elcius': 59, 'Johannes': 41}
Valid transaction: {'Elcius': -1, 'Johannes': 1},  State: {'Elcius': 58, 'Johannes': 42}
Valid transaction: {'Elcius': 1, 'Johannes': -1},  State: {'Elcius': 59, 'Johannes': 41}
Valid transaction: {'Elcius': 1, 'Johannes': -1},  State: {'Elcius': 60, 'Johannes': 40}
Valid transaction: {'Elcius': -3, 'Johannes': 3},  State: {'Elcius': 57, 'Johannes': 43}
Valid transaction: {'Elcius': 2, 'Johannes': -2},  State: {'Elcius': 59, 'Johannes': 41}
Valid transaction: {'Elcius': 2, 'Johannes': -2},  State: {'Elcius': 61, 'Johannes': 39}
Valid transaction: {'Elcius': -2, 'Johannes': 2},  State: {'Elcius': 59, 'Johannes': 41}
Valid transaction: {'

#### Como resultado temos a seguinte Blockchain:

In [308]:
print(json.dumps(chain, indent=3))

[
   {
      "hash": "b1f3f3cf7fb886cffc1e4b61cf65a74d18d2ee8b85ee80db26a8b0e6e2795ef5",
      "content": {
         "block_number": 0,
         "parent_hash": null,
         "trns_quantity": 1,
         "transactions": [
            {
               "Elcius": 50,
               "Johannes": 50
            }
         ]
      }
   },
   {
      "hash": "269bfdc756f46e85dfba8f6f7601237908ebb1494e8875c36d2fe9b3cfe53758",
      "content": {
         "block_number": 1,
         "parent_hash": "b1f3f3cf7fb886cffc1e4b61cf65a74d18d2ee8b85ee80db26a8b0e6e2795ef5",
         "trns_quantity": 5,
         "transactions": [
            {
               "Elcius": 3,
               "Johannes": -3
            },
            {
               "Elcius": 3,
               "Johannes": -3
            },
            {
               "Elcius": 2,
               "Johannes": -2
            },
            {
               "Elcius": 1,
               "Johannes": -1
            },
            {
               "Elcius

In [309]:
state

{'Elcius': 64, 'Johannes': 36}

## Validação da cadeia

#### Agora que é possível criar blocos e adicioná-los à cadeia, é necessário realizar uma validação para os novos blocos e consequentemente para a cadeia inteira.

In [310]:
''' Helper function to verify if the block hash matches it's content. 
    If it does not match then raises an exception'''
def check_block_hash(block):
    expected_hash = hashing(block['content'])
    if expected_hash != block['hash']:
        raise Exception('Hash does not match contents of block %s' % block['contents']['block_number'])
    return

#### A função a seguir verifica se: 
* Cada transação é válida dado o estado atual
* O hash do bloco é válido de acordo com seu conteúdo
* O número do bloco é igual ao número do bloco anterior + 1
* Se refere corretamente ao hash do bloco anterior

In [311]:
''' Check if the block is valid using information about parent and current state '''
def check_block_validity(block, parent, state):
    error = False
    parent_number = parent['content']['block_number']
    parent_hash = parent['hash']
    block_number = block['content']['block_number']
    
    # Check transactions validity
    for trn in block['content']['transactions']:
        if transaction_validation(trn, state): 
            state = update_state(trn, state)
        else:
            raise Exception('Invalid transaction in block %s: %s' % (block_number, trn))
            
    # Check content integrity
    if check_block_hash(block) == False:
        print('Hash does not match content of block %s' % block['content']['block_number'])
        error = True
        return state, error
    
    # Check block number
    if block_number != (parent_number + 1):
        raise Exception('Parent number/block number not accurate at block %s' % block_number)
    
    # Check parent hash
    if block['content']['parent_hash'] != parent_hash:
        raise Exception('Parent hash not accurate at block %s' % block_number)
    
    return state

#### A função abaixo percorre a cadeia a partir do bloco gênesis (que tem tratamento diferenciado) verificando se todas as transações são válidas e se os blocos estão conectados pelos hashes.

In [313]:
''' Check the validity of the entire chain and computes the state starting from the genesis block.
    Returns the state if it is a valid chain, else prints an error messsage'''
def check_chain(chain):
    # Data input processing
    if type(chain) == str:
        try:
            chain = json.loads(chain)
            assert(type(chain) == list)   # Make sure the chain is a list of dicts
        except: 
            return False
    elif type(chain) != list:
        return False
    
    state = {}
    
    # Creating genesis block
    for trn in chain[0]['content']['transactions']:
        state = update_state(trn, state)
    check_block_hash(chain[0])
    parent = chain[0]
    
    # Checking next blocks
    for block in chain[1:]:
        state = check_block_validity(block, parent, state)
        parent = block
        
    return state

In [314]:
check_chain(chain)

{'Elcius': 64, 'Johannes': 36}

## Arquitetura final

#### Em uma rede real de uma Blockchain, novos nós baixam uma cópia da cadeia e fazem o que acabamos de fazer acima, verificam ela. Conforme mineradores criam novos blocos (pegando transações do buffer, validando-as e juntando-as), cada bloco novo chega aos nós da rede para que cada um adicione este bloco recém criado em suas próprias cópias da Blockchain.

#### Como já sabemos reunir transações em um bloco e verificar a cópia da Blockchain, se simularmos o recebimento de um novo bloco, podemos verificá-lo e adicioná-lo na nossa cadeia.

#### Representando o trabalho de um minerador de maneira simplificada temos:

In [316]:
import copy

miner_node_chain = copy.copy(chain)   # miner copies the Blockchain 
miner_node_trns = [transaction_generator() for i in range(5)]   #  get transactions
new_block = make_block(miner_node_trns, miner_node_chain)   # create a block
new_block

{'content': {'block_number': 4,
  'parent_hash': '660c4526071fb12cb0bd4c4b450e673c7f501bff420cb19eb51f67d803767eaf',
  'transactions': [{'Elcius': 3, 'Johannes': -3},
   {'Elcius': 1, 'Johannes': -1},
   {'Elcius': -1, 'Johannes': 1},
   {'Elcius': 1, 'Johannes': -1},
   {'Elcius': 3, 'Johannes': -3}],
  'trns_quantity': 5},
 'hash': 'b4644bcaee4d7722a925e88fe6518b854840cc9c18e8b306b85d27b59d0a6f24'}

#### Agora, assumimos que esse novo bloco minerado é transmitido para nosso nó pela rede. Verificaremos se esse bloco é válido, caso seja será adicionado à copia da Blockchain que temos.

In [317]:
print("Current state: ", state)
print("Blockchain on our node is currently %s blocks long\n" % len(chain))

try:
    print("New block received...")
    print("Checking validity...\n")
    # Update the state. Throw an error if the block is invalid.
    state = check_block_validity(new_block, chain[-1], state)
    chain.append(new_block)
    print("Valid block! Waiting for the next block...\n")
except:
    print("Invalid block! Ignoring it and waiting for the next block...\n")

print("New current state: ", state)
print("Blockchain on Node A is now %s blocks long." % len(chain))

Current state:  {'Elcius': 64, 'Johannes': 36}
Blockchain on our node is currently 4 blocks long

New block received...
Checking validity...

Valid block! Waiting for the next block...

New current state:  {'Elcius': 71, 'Johannes': 29}
Blockchain on Node A is now 5 blocks long.


## Conclusão

#### Desta forma têm-se a arquitetura básica de uma Blockchain, da criação de transações simplificadas à mecanismos de validação das mesmas, criação e validação de blocos e da cadeia inteira. Pode-se simular a manipulação dos saldos das contas a partir de uma cópia baixada da Blockchain, simular o recebimento de um novo bloco pela rede, validá-lo ou criá-lo.

#### Não foram abordados alguns fatores da arquitetura da rede e conceitos como proof-of-work, consensus mechanism e criptografia (que garantem à Blockchain segurança contra ataques) também não foram explorados. Elementos que ficaram para um estudo futuro mais aprofundado!